In [1]:
import os
import torch 
import numpy
import pandas as pd 
import cudf
import cupy
import nvtabular as nvt

### Create random input data similar to pre-processed Yoochoose dataset structure 

In [2]:
NUM_ROWS = 200000
session_length = 20
inputs = {
    'session_id': numpy.random.randint(70000, 80000, NUM_ROWS),
    'day': numpy.random.randint(1, 10, NUM_ROWS),
    'item_id': numpy.random.randint(1, 51996, NUM_ROWS),
    'category': numpy.random.randint(0, 332, NUM_ROWS),
    'timestamp/age_days': numpy.random.uniform(0, 1, NUM_ROWS),
    'timestamp/weekday/sin' : numpy.random.uniform(0, 1, NUM_ROWS),
    'purchase': numpy.random.randint(0, 2, NUM_ROWS)
    }
random_data = cudf.DataFrame(inputs)
random_data

,session_id,day,item_id,category,timestamp/age_days,timestamp/weekday/sin,purchase
0,78657,3,14220,200,0.042894,0.980474,0
1,74690,5,45798,111,0.105155,0.009693,0
2,74941,1,27810,176,0.402443,0.858032,0
3,72915,8,15592,197,0.791257,0.507413,0
4,79719,4,12723,233,0.302400,0.547591,1
...,...,...,...,...,...,...,...
199995,74677,1,48906,245,0.801508,0.408195,1
199996,76836,2,45562,99,0.677022,0.951179,0
199997,78828,5,19379,325,0.226396,0.329395,0
199998,74510,5,12348,85,0.598994,0.394180,1


### NVTabular workflow

- #TODO : Change the workflow using tagging API once it is finalized  

In [3]:
# Define Groupby Workflow
groupby_features = list(inputs.keys()) >> nvt.ops.Groupby(
    groupby_cols=["session_id"], 
    aggs={
        "item_id": ["list"],
        "category": ["list"],     
        "day": ["first"],
        "purchase": ["first"],
        "timestamp/age_days": ["list"],
        'timestamp/weekday/sin': ["list"],
        },
    name_sep="-")
# Trim sessions to first 20 items 
groupby_features_list = [x for x in groupby_features.output_columns.names if '-list' in x]

#groupby_features_nonlist:  need to fix a BUG related to adding two workflow nodes
#groupby_features_trim = groupby_features_list >> nvt.ops.ListSlice(0,20) >> nvt.ops.Rename(postfix = '_trim')
workflow = nvt.Workflow(groupby_features)
dataset = nvt.Dataset(random_data, cpu=False)
workflow.fit(dataset)
sessions_gdf = workflow.transform(dataset).to_ddf().compute()

# Re-compute tri: to be removed when the BUG of two workflow nodes is fixed : 
groupby_features_trim =  groupby_features_list >> nvt.ops.ListSlice(0,20) >> nvt.ops.Rename(postfix = '_trim')
workflow = nvt.Workflow(list(sessions_gdf.columns)  + groupby_features_trim)
dataset = nvt.Dataset(sessions_gdf, cpu=False)
workflow.fit(dataset)
sessions_gdf = workflow.transform(dataset).to_ddf().compute()

In [4]:
sessions_gdf.head(3)

,timestamp/age_days-list_trim,timestamp/weekday/sin-list_trim,item_id-list_trim,category-list_trim,session_id,timestamp/age_days-list,timestamp/weekday/sin-list,day-first,item_id-list,purchase-first,category-list
0,"[0.2159907365849888, 0.38978614558809666, 0.41...","[0.34691455014776873, 0.6141594448562755, 0.19...","[3453, 29113, 40598, 18312, 17358, 16541, 2983...","[146, 88, 231, 289, 43, 228, 243, 259, 10, 278...",70000,"[0.2159907365849888, 0.38978614558809666, 0.41...","[0.34691455014776873, 0.6141594448562755, 0.19...",6,"[3453, 29113, 40598, 18312, 17358, 16541, 2983...",0,"[146, 88, 231, 289, 43, 228, 243, 259, 10, 278..."
1,"[0.16954246605001888, 0.7339434412221009, 0.54...","[0.29087761083519337, 0.23826932571806037, 0.2...","[17305, 47030, 9157, 3953, 39788, 40392, 3221,...","[154, 2, 297, 168, 140, 38, 280, 174, 118, 249...",70001,"[0.16954246605001888, 0.7339434412221009, 0.54...","[0.29087761083519337, 0.23826932571806037, 0.2...",7,"[17305, 47030, 9157, 3953, 39788, 40392, 3221,...",0,"[154, 2, 297, 168, 140, 38, 280, 174, 118, 249..."
2,"[0.7209095602846058, 0.11445547315539895, 0.34...","[0.43690866483674196, 0.6574986542124382, 0.49...","[10705, 7155, 40486, 6641, 25219, 8900, 42472,...","[51, 330, 314, 76, 271, 274, 265, 231, 78, 325...",70002,"[0.7209095602846058, 0.11445547315539895, 0.34...","[0.43690866483674196, 0.6574986542124382, 0.49...",9,"[10705, 7155, 40486, 6641, 25219, 8900, 42472,...",0,"[51, 330, 314, 76, 271, 274, 265, 231, 78, 325..."


- We can save the workflow.

In [5]:
workflow.save('workflow_inference_test')

### Export pre-processed data by day 

In [6]:
# Convert to a Dataset and write out hive-partitioned data to disk
nvt_output_path_tmp ='./output_nvt_tmp/'
PARTITION_COL = 'day-first'
nvt.Dataset(sessions_gdf).to_parquet(nvt_output_path_tmp, partition_on=[PARTITION_COL])

In [7]:
OUTPUT_FOLDER = "./preproc_sessions_by_day_ts/"
!mkdir -p $OUTPUT_FOLDER
days_folders = [f for f in sorted(os.listdir(nvt_output_path_tmp)) if f.startswith(PARTITION_COL)]
for day_folder in days_folders:
    df = cudf.read_parquet(os.path.join(nvt_output_path_tmp, day_folder))
    out_folder = os.path.join(OUTPUT_FOLDER, day_folder.replace('day-first=', ''))
    os.makedirs(out_folder, exist_ok=True)
    df.to_parquet(os.path.join(out_folder, 'train.parquet'))
    
    random_values = cupy.random.rand(len(df))
    
    #Extracts 10% for valid and test set. Those sessions are also in the train set, but as evaluation
    #happens only for the subsequent day of training, that is not an issue, and we can keep the train set larger.
    valid_set = df[random_values <= 0.10]
    valid_set.to_parquet(os.path.join(out_folder, 'valid.parquet'))
    
    test_set = df[random_values >= 0.90]
    test_set.to_parquet(os.path.join(out_folder, 'test.parquet'))

# Transformers4rec model  

In [8]:
import torch 
import transformers4rec.torch as torch4rec
from transformers4rec.torch import TabularSequenceFeatures, MLPBlock, SequentialBlock, Head, TransformerBlock

from transformers4rec.utils.schema import DatasetSchema
from transformers4rec.torch.head import NextItemPredictionTask
from transformers4rec.config import transformer
from transformers4rec.torch.ranking_metric import NDCGAt, AvgPrecisionAt, RecallAt

NameError: name 'Optional' is not defined

- Manually set the schema 

In [ ]:
# Define schema object to pass it to the SequentialTabularFeatures
schema = DatasetSchema.from_schema("schema.pb")

### Define the sequential input module

Below we define our `input` bloc using [`SequentialTabularFeatures` class](https://github.com/NVIDIA-Merlin/Transformers4Rec/blob/main/transformers4rec/torch/features/sequential.py). The `from_schema` module directly parse schema and accepts categorical and continuous sequential inputs and supports data augmentation, data aggregation, `sequential-concat` and `elementwise-sum` aggregations, the projection of the interaction embeddings and the masking tasks.

`max_sequence_length` defines the maximum sequence length of our sequential input, and if `continuous_projection` argument is set,  all numerical features are concatenated and projected by a number of MLP layers.

In [ ]:
inputs = TabularSequenceFeatures.from_schema(
        schema,
        max_sequence_length=20,
        continuous_projection=64,
        d_output=100,
        masking="causal",
    )

### End-to-end session-based Transformer-based model for item prediction:

- LM task + HF Transformer architecture + Next item-prediction task. 
- We build a [T4RecConfig](https://github.com/NVIDIA-Merlin/Transformers4Rec/blob/main/transformers4rec/config/transformer.py#L8) class to update the config class of the transformer architecture with the specified arguments, then load the related model. Here we use it to instantiate an XLNET model according to the  arguments (d_model, n_head, etc.), defining the model architecture.
- [TransformerBlock](https://github.com/NVIDIA-Merlin/Transformers4Rec/blob/main/transformers4rec/torch/block/transformer.py#L37) class is created to support HF Transformers for session-based and sequential-based recommendation models.
- [NextItemPredictionTask](https://github.com/NVIDIA-Merlin/Transformers4Rec/blob/main/transformers4rec/torch/head.py#L212) is the class to support next item prediction task.

In [ ]:
#### Define XLNetConfig class and set default parameters 

# Set HF config of XLNet 
transformer_config = transformer.XLNetConfig.build(
    d_model=64, n_head=4, n_layer=2, total_seq_length=20
)
# Define the model block including: inputs, masking, projection and transformer block.
body = torch4rec.SequentialBlock(
    inputs, torch4rec.MLPBlock([64]), torch4rec.TransformerBlock(transformer=transformer_config, masking=inputs.masking)
)

# Define the head related to next item prediction task 
head = torch4rec.Head(
    body,
    torch4rec.NextItemPredictionTask(weight_tying=True, hf_format=True),
    inputs=inputs,
    
)

# Get the end-to-end Model class 
model = torch4rec.Model(head, device='cuda')

# Non-incremental training and evaluation

- **Set Training arguments**

In [ ]:
from transformers4rec.config.trainer import T4RecTrainingArguments
from transformers4rec.torch import Trainer
#Set argumentd for training 
train_args = T4RecTrainingArguments(local_rank = -1, dataloader_drop_last = True, engine='nvtabular',
                                  report_to = [], debug = ["r"], gradient_accumulation_steps = 32,
                                  per_device_train_batch_size = 512, per_device_eval_batch_size = 32,
                                  output_dir = ".", use_legacy_prediction_loop = False,
                                  max_sequence_length=20)

- **Define paths to train and eval data**

In [ ]:
import glob
from transformers4rec.torch.utils.data_utils import PyarrowDataLoaderBuilder, NVTDataLoaderBuilder
train_data_paths = glob.glob("./preproc_sessions_by_day_ts/*/train.parquet")[:-1]
eval_data_paths = glob.glob("./preproc_sessions_by_day_ts/*/valid.parquet")[:-1]

- **Define Trainer**

In [ ]:
# Instantiate the T4Rec Trainer, which manages training and evaluation
trainer = Trainer(
    model=model,
    args=train_args,
    schema=schema,
    compute_metrics=True,
    train_dataset_or_path=train_data_paths,
    eval_dataset_or_path=eval_data_paths,
)

- **Train the model**  

In [ ]:
trainer.reset_lr_scheduler()
trainer.train()

- **Compute evaluation metrics**

In [ ]:
eval_metrics = trainer.evaluate(eval_dataset=eval_data_paths, metric_key_prefix='eval')
for key in sorted(eval_metrics.keys()):
    print("  %s = %s" % (key, str(eval_metrics[key])))

- **Compute Train metrics**

In [ ]:
train_metrics = trainer.evaluate(eval_dataset=train_data_paths, metric_key_prefix='train')
for key in sorted(train_metrics.keys()):
    print("  %s = %s" % (key, str(train_metrics[key])))

* **Save the model**

In [ ]:
trainer._save_model_and_checkpoint()

* **Reload model:**

In [ ]:
trainer.load_model_trainer_states_from_checkpoint('./checkpoint-3')

- **Re-compute eval metrics of train data**

In [ ]:
train_metrics = trainer.evaluate(eval_dataset=train_data_paths, metric_key_prefix='train')
for key in sorted(train_metrics.keys()):
    print("  %s = %s" % (key, str(train_metrics[key])))

* **Resume Training** 

In [ ]:
# reset lr scheduler to train on new day data
trainer.reset_lr_scheduler()
# set new data from last day
trainer.train_dataset = train_data_paths[-1]
trainer.train(resume_from_checkpoint='./checkpoint-3')

- **Evaluate on last day**

In [ ]:
# set new data from last day
eval_metrics = trainer.evaluate(eval_dataset=eval_data_paths[-1], metric_key_prefix='eval')
for key in sorted(eval_metrics.keys()):
    print("  %s = %s" % (key, str(eval_metrics[key])))